In [ ]:
#%env KERAS_BACKEND=theano
%env PYTHONHASHSEED=0
from tensorflow import set_random_seed
set_random_seed(1234)
import numpy as np
np.random.seed(1234)
import random as rn
rn.seed(1234)


import pandas as pd

import matplotlib.pyplot as plt
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import log_loss
from kaggle.competitions import twosigmanews


#***********************************import keras
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.utils import conv_utils
from keras.utils.data_utils import get_file
from keras.engine.topology import get_source_inputs
from keras.engine import InputSpec
from keras import backend as K
from keras.layers import LeakyReLU
from keras.layers import ZeroPadding2D
from keras.losses import binary_crossentropy
import keras.callbacks as callbacks
from keras.callbacks import Callback
from keras.applications.xception import Xception
from keras.layers import multiply

import keras
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.engine.topology import Input
from keras.engine.training import Model
from keras.layers.convolutional import Conv2D, UpSampling2D, Conv2DTranspose
from keras.layers.core import Activation, SpatialDropout2D
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add
from keras.regularizers import l2
from keras.layers.core import Dense, Lambda
from keras.layers.merge import concatenate, add
from keras.layers import GlobalAveragePooling2D, Reshape, Dense, multiply, Permute
from keras.optimizers import SGD

In [ ]:
env = twosigmanews.make_env()
(market_train, _) = env.get_training_data()
print(market_train.shape)

In [ ]:
currentValDicts = {}
import math

jj = 0

 
def getEMWA(assetCode, alpha, val, name):
    if math.isnan(val):
        val = 0
    
    try:
        currentValDict = currentValDicts[str(alpha) + name]
    except KeyError:
        currentValDict = {}
        currentValDicts[str(alpha) + name] = currentValDict
        
    try:
        currentValue = currentValDict[assetCode]
    except KeyError:
        currentValue = 0
        
    result = alpha * val + (1-alpha) * currentValue
    currentValDict[assetCode] = result
    return result

assetCodeStats = {}
def getStats(assetCode, next10mktresVal):
    if math.isnan(next10mktresVal):
        next10mktresVal = 0
        
    try:
        currentStats = assetCodeStats[assetCode]
    except KeyError:
        currentStats = {"totalCount": 0, "positives" : 0}
        assetCodeStats[assetCode] = currentStats
        
    currentStats['totalCount'] = currentStats['totalCount'] + 1
    currentStats['positives'] = currentStats['positives'] + (1 if next10mktresVal > 0 else 0)
    
    return currentStats
    

In [ ]:
plt.hist(market_train[market_train.time == '2007-02-01 22:00:00+00:00'].returnsOpenNextMktres10, bins=100)

In [ ]:
plt.hist(market_train[market_train.time == '2015-12-03 22:00:00+00:00'].returnsOpenNextMktres10, bins=100)

In [ ]:
market_train.groupby('time')['returnsOpenNextMktres10'].agg(lambda x: (x>0).sum() / ((x>0).sum() + (x<0).sum()))

In [ ]:
mytimestep=20

train_indices=np.array([i for i in range(800, 1800)]) 
train_indices_timestep=np.array([i for i in range(800 + mytimestep - 1, 1800)]) 
val_indices=np.array([i for i in range(1810, 2490)])
val_indices_timestep=np.array([i for i in range(1810 + mytimestep - 1, 2490)])
latest100_indices = np.array([i for i in range(2460, 2490)]) 
latest_indices = np.array([i for i in range(0, 2498)]) 
all_indices=np.array([i for i in range(1, 4072955)]) 

In [ ]:
from sklearn.preprocessing import scale
def enrich_df(_df):
    _df.loc[_df['returnsOpenPrevMktres10']>1, 'returnsOpenPrevMktres10'] = 1
    _df.loc[_df['returnsOpenPrevMktres10']<-1, 'returnsOpenPrevMktres10'] = -1
    _df.loc[_df['returnsClosePrevMktres10']>1, 'returnsClosePrevMktres10'] = 1
    _df.loc[_df['returnsClosePrevMktres10']<-1, 'returnsClosePrevMktres10'] = -1
    _df.loc[_df['returnsOpenPrevMktres1']>0.3, 'returnsOpenPrevMktres1'] = 0.3
    _df.loc[_df['returnsOpenPrevMktres1']<-0.3, 'returnsOpenPrevMktres1'] = -0.3
    _df.loc[_df['returnsClosePrevMktres1']>0.3, 'returnsClosePrevMktres1'] = 0.3
    _df.loc[_df['returnsClosePrevMktres1']<-0.3, 'returnsClosePrevMktres1'] = -0.3
    _df.loc[_df['returnsOpenPrevRaw10']>0.7, 'returnsOpenPrevRaw10'] = 0.7
    _df.loc[_df['returnsOpenPrevRaw10']<-0.7, 'returnsOpenPrevRaw10'] = -0.7
    _df.loc[_df['returnsOpenPrevRaw1']>0.5, 'returnsOpenPrevRaw1'] = 0.5
    _df.loc[_df['returnsOpenPrevRaw1']<-0.5, 'returnsOpenPrevRaw1'] = -0.5

    _df['market_code'] = _df.assetCode.str.split('.', expand=True)[1]

    _df['returnsOpenPrevMktres10SO'] = _df['returnsOpenPrevMktres10'].transform(lambda x: 1 if x > 0 else 0)
    _df['returnsClosePrevMktres10SO'] = _df['returnsClosePrevMktres10'].transform(lambda x: 1 if x > 0 else 0)
    _df['returnsOpenPrevMktres1SO'] = _df['returnsOpenPrevMktres1'].transform(lambda x: 1 if x > 0 else 0)

    # for i in _df.index:
    #     assetCode = _df.at[i, 'assetCode']
    # #     _df.at[i, 'returnsOpenPrevMktres10_emwa033_custom'] = getEMWA(assetCode, 0.33,  _df.at[i, 'returnsOpenPrevMktres10'], 'returnsOpenPrevMktres10')
    # #     _df.at[i, 'returnsOpenPrevMktres10_emwa014_custom'] = getEMWA(assetCode, 0.14,  _df.at[i, 'returnsOpenPrevMktres10'], 'returnsOpenPrevMktres10')
    # #     _df.at[i, 'returnsOpenPrevMktres10_emwa007_custom'] = getEMWA(assetCode, 0.07,  _df.at[i, 'returnsOpenPrevMktres10'], 'returnsOpenPrevMktres10')
    #     assetStats =  getStats(assetCode,  _df.at[i, 'returnsOpenPrevMktres10'])
    #     _df.at[i, 'assetCodePerfC'] = assetStats['positives'] /  assetStats['totalCount']
    #     _df.at[i, 'assetCodePerfCountC'] = assetStats['totalCount']

    _df['returnsOpenPrevMktres10_dailyMean'] = _df.groupby('time')['returnsOpenPrevMktres10'].mean()
    _df['returnsClosePrevMktres10_dailyMean'] = _df.groupby('time')['returnsClosePrevMktres10'].mean()
    _df['returnsOpenPrevMktres1_dailyMean'] = _df.groupby('time')['returnsOpenPrevMktres1'].mean()
    _df['returnsClosePrevMktres1_dailyMean'] = _df.groupby('time')['returnsClosePrevMktres1'].mean()
    _df['returnsOpenPrevRaw10_dailyMean'] = _df.groupby('time')['returnsOpenPrevRaw10'].mean()
    _df['returnsOpenPrevRaw1_dailyMean'] = _df.groupby('time')['returnsOpenPrevRaw1'].mean()

    #_df['returnsOpenPrevMktres10_dailyMean_market'] =  _df.groupby(['time', 'market_code'])['returnsOpenPrevMktres10'].transform('mean')
    _df['returnsClosePrevMktres10_dailyMean_market'] =  _df.groupby(['time', 'market_code'])['returnsClosePrevMktres10'].transform('mean')
    #_df['returnsOpenPrevMktres1_dailyMean_market'] =  _df.groupby(['time', 'market_code'])['returnsOpenPrevMktres1'].transform('mean')
    _df['returnsClosePrevMktres1_dailyMean_market'] =  _df.groupby(['time', 'market_code'])['returnsClosePrevMktres1'].transform('mean')
    #_df['returnsOpenPrevRaw10_dailyMean_market'] =  _df.groupby(['time', 'market_code'])['returnsOpenPrevRaw10'].transform('mean')
    _df['returnsClosePrevRaw10_dailyMean_market'] =  _df.groupby(['time', 'market_code'])['returnsClosePrevRaw10'].transform('mean')
    _df['returnsOpenPrevRaw1_dailyMean_market'] =  _df.groupby(['time', 'market_code'])['returnsOpenPrevRaw1'].transform('mean')

    _df['returnsOpenPrevMktres10SO_dailyMean'] =  _df.groupby('time')['returnsOpenPrevMktres10SO'].transform('mean')
    _df['returnsOpenPrevMktres10SO_dailyMean_market'] =  _df.groupby(['time', 'market_code'])['returnsOpenPrevMktres10SO'].transform('mean')
    _df['returnsClosePrevMktres10SO_dailyMean_market'] =  _df.groupby(['time', 'market_code'])['returnsClosePrevMktres10SO'].transform('mean')
    _df['returnsOpenPrevMktres1SO_dailyMean_market'] =  _df.groupby(['time', 'market_code'])['returnsOpenPrevMktres1SO'].transform('mean')
    
    print('unique time', _df['time'].nunique())
    print('unique market_code', _df['market_code'].unique())
    return _df

market_train = enrich_df(market_train)


In [ ]:
pd.options.display.max_rows = 300
pd.DataFrame(market_train.groupby(['time', 'market_code'])['returnsOpenPrevMktres1SO'].agg('mean'))


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
#volumeScaler = StandardScaler()
#market_train['volume'] = market_train.groupby('assetCode')['volume'].transform(lambda x: scale(x.astype(float)))
#market_train[['returnsOpenPrevMktres10', 'returnsOpenPrevMktres1', 'volume' ,'returnsOpenPrevRaw10', 'returnsOpenPrevMktres10_emwa033_custom', 'returnsOpenPrevMktres10_emwa014_custom', 'returnsOpenPrevMktres10_emwa007_custom']] = scaler.fit_transform(market_train[['returnsOpenPrevMktres10', 'returnsOpenPrevMktres1', 'volume', 'returnsOpenPrevRaw10', 'returnsOpenPrevMktres10_emwa033_custom', 'returnsOpenPrevMktres10_emwa014_custom', 'returnsOpenPrevMktres10_emwa007_custom']])
#market_train[['returnsOpenPrevMktres10', 'returnsOpenPrevMktres1', 'volume' ,'returnsOpenPrevRaw10', 'assetCodePerf' , 'assetCodePerfCount', 'assetCodePerfC' , 'assetCodePerfCountC']] = scaler.fit_transform(market_train[['returnsOpenPrevMktres10', 'returnsOpenPrevMktres1', 'volume', 'returnsOpenPrevRaw10', 'assetCodePerf', 'assetCodePerfCount', 'assetCodePerfC' , 'assetCodePerfCountC']])


#train_X_reshaped[:,:,0] = scaler.fit_transform(train_X_reshaped)
#val_X_reshaped = scaler.transform(val_X_reshaped)

In [ ]:
cat_cols = ['market_code'] 
num_cols = [
              'volume', 
              'returnsOpenPrevMktres1', 'returnsClosePrevMktres1', 
              'returnsOpenPrevMktres10',  'returnsClosePrevMktres10',
       
              'returnsOpenPrevRaw10', 'returnsClosePrevRaw10', 
              'returnsOpenPrevRaw1', 'returnsClosePrevRaw1',
#              'close', 'open',
#               'returnsOpenPrevMktres10_dailyMean', #'returnsClosePrevMktres10_dailyMean', 
#               'returnsOpenPrevMktres1_dailyMean', #'returnsClosePrevMktres1_dailyMean', 
#               'returnsOpenPrevRaw10_dailyMean', #'returnsOpenPrevRaw1_dailyMean',
#               'returnsOpenPrevMktres10SO_dailyMean',
    
              #'returnsOpenPrevMktres10_dailyMean_market', 
               'returnsClosePrevMktres10_dailyMean_market', 
              #'returnsOpenPrevMktres1_dailyMean_market', 
              'returnsClosePrevMktres1_dailyMean_market', 
              #'returnsOpenPrevRaw10_dailyMean_market', 
               'returnsClosePrevRaw10_dailyMean_market',
#              'returnsOpenPrevRaw1_dailyMean_market',
              #'returnsOpenPrevMktres10SO_dailyMean_market',
               'returnsOpenPrevMktres10SO_dailyMean_market',
#              'returnsOpenPrevMktres1SO_dailyMean_market',
#              'assetCodePerfC', 'assetCodePerfCountC',
              #'returnsOpenPrevMktres10SO'
              'returnsClosePrevMktres10SO'

           ]



In [ ]:
print('reindex data...')
all_assetCodes = market_train.assetCode.unique()
market_train = market_train.set_index(['time', 'assetCode'], drop=False)
(time_index, asset_index) = market_train.index.levels
new_index = pd.MultiIndex.from_product([time_index, all_assetCodes], names=['time', 'assetCode'])
market_train = market_train.reindex(new_index)
print(market_train.shape)

In [ ]:
market_train[market_train.assetCode=='AAPL.O'].plot(x='time', y=['returnsOpenPrevMktres10','returnsOpenPrevMktres1', 'volume'] , figsize=(150,10))
market_train[market_train.assetCode=='AAPL.O'].plot(x='time', y=['returnsOpenPrevMktres10','returnsOpenNextMktres10'] , figsize=(150,10))

# market_train[market_train.assetCode=='AAPL.O'].plot(x='time', y=['assetCodePerfC'] , figsize=(150,10))
# market_train[market_train.assetCode=='AAP.N'].plot(x='time', y=['assetCodePerfC'] , figsize=(150,10))

In [ ]:
#market_train = market_train[market_train.assetCode=='AAPL.O']
#market_train = market_train[market_train.assetCode=='AAP.N']

from pandas.tools.plotting import autocorrelation_plot
autocorrelation_plot(market_train[market_train.assetCode=='AAPL.O']['returnsOpenPrevMktres10'])
plt.show()

from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(market_train[market_train.assetCode=='AAPL.O']['returnsOpenPrevMktres10'].iloc[10:], order=(7,0,0))
model_fit = model.fit(disp=0)
print(model_fit.summary())
model_fit.predict(start=market_train[market_train.assetCode=='AAPL.O']['returnsOpenPrevMktres10'].iloc[10:].shape[0], end=market_train[market_train.assetCode=='AAPL.O']['returnsOpenPrevMktres10'].iloc[10:].shape[0]+10)

In [ ]:
def encode(encoder, x):
    len_encoder = len(encoder)
    try:
        id = encoder[x]
    except KeyError:
        id = len_encoder
    return id

encoders = [{} for cat in cat_cols]


for i, cat in enumerate(cat_cols):
    print('encoding %s ...' % cat, end=' ')
    #encoders[i] = {l: id for id, l in enumerate(market_train.loc[train_indices, cat].astype(str).unique())}
    encoders[i] = {l: id for id, l in enumerate(market_train.loc[:, cat].astype(str).unique())}
    market_train[cat+"_encoded"] = market_train[cat].astype(str).apply(lambda x: encode(encoders[i], x))
    print('Done')

embed_sizes = [len(encoder) + 1 for encoder in encoders] #+1 for possible unknown assets

In [ ]:
market_train[market_train.index.get_level_values(1)=='AAPL.O'][['returnsOpenPrevMktres10', 'returnsOpenPrevMktres1', 'returnsOpenNextMktres10']]

In [ ]:
def getMyData(_df, _days=2498):
    mydata = np.zeros((_days, _df.index.get_level_values(1).nunique(), len(num_cols)))
    print('mydata.shape', mydata.shape)
    
    nn = 0
    for num_col in num_cols:
        #print('num_col', num_col)
        _mylist =_df.groupby([_df.index.get_level_values(0)])[num_col].apply(lambda x : x.values.tolist())
        _myarr = np.array(_mylist)
        _myarr = np.array([i for i in _myarr])
        mydata[:,:,nn] = _myarr
        nn = nn + 1

    #mydata = np.reshape(mydata, (_days, -1)) #2498 days
    print('mydata', mydata.shape)
    return mydata
mydata = getMyData(market_train)

_myylist =market_train.groupby([market_train.index.get_level_values(0)])['returnsOpenNextMktres10'].apply(lambda x : x.values.tolist())
_myyarr = np.array(_myylist)
_myyarr = np.array([i for i in _myyarr])
myy = np.expand_dims(_myyarr, axis=2)
print('myy', myy.shape)

_myulist =market_train.groupby([market_train.index.get_level_values(0)])['universe'].apply(lambda x : x.values.tolist())
_myuarr = np.array(_myulist)
_myuarr = np.array([i for i in _myuarr])
myu = np.expand_dims(_myuarr, axis=2)
print('myu', myu.shape)

days = market_train.index.get_level_values(0)
days = days.unique()
myd = np.array([days for d in range(0, 3780)])
myd = myd.T
myd = np.expand_dims(myd, axis=2)

del _myylist, _myyarr, _myulist, _myuarr, days
import gc
n = gc.collect()
print('gc collect called ', n)

print('myd', myd.shape)
print('done reindexing')

In [ ]:
asset_corr = np.corrcoef(np.nan_to_num(mydata[:,:,4]).T)
pd.DataFrame(data=asset_corr)

In [ ]:
asset_corr[asset_corr>0.999] = 0
pd.DataFrame(data=[np.max(np.nan_to_num(asset_corr), axis=0), np.argmax(np.nan_to_num(asset_corr), axis=0)])

In [ ]:
market_train[market_train.assetCode=='AAPL.O'].plot(x='time', y=['returnsOpenPrevMktres10','returnsOpenPrevMktres1', 'volume'] , figsize=(150,10))

In [ ]:
np.set_printoptions(edgeitems=20)
print('mydata AAPL.O', mydata[:,3]) #3 is AAPL.O
np.set_printoptions(edgeitems=3)

# Handling categorical variables

# Define NN Architecture

Todo: add explanaition of architecture

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Embedding, Concatenate, Flatten, BatchNormalization, LSTM, GRU, Masking, RepeatVector
from keras.losses import binary_crossentropy, mse
from keras import regularizers



def getLSTMModel():
    inputs = Input(shape=(mytimestep, len(num_cols),))

    #assetCodeInput = Input(shape=(mytimestep, 1,), name="assetCodeInput")
    #assetCodeEmbedding = Embedding(3444, 7, embeddings_regularizer=regularizers.L1L2(l1=0.1, l2=0.1))(assetCodeInput)
    #assetCodeEmbedding = Reshape((mytimestep, 7))(assetCodeEmbedding)

    #concatInputs = Concatenate()([inputs, assetCodeEmbedding])

#    masked = Masking()(inputs)
    #x = LSTM(10, stateful=False,  return_sequences=True,   recurrent_dropout=0., kernel_regularizer=regularizers.L1L2(l1=0.0, l2=0.0))(inputs)
    #x = Dropout(0.5)(x)
#     x = LSTM(10,  stateful=False,  return_sequences=True,  recurrent_dropout=0., kernel_regularizer=regularizers.L1L2(l1=0.0, l2=0.0))(x)
#     x = Dropout(0.5)(x)
    x = LSTM(10, stateful=False,  return_sequences=False,  recurrent_dropout=0., kernel_regularizer=regularizers.L1L2(l1=0.0, l2=0.0))(inputs)
    x = Dropout(0.8)(x)
    
#     x = LSTM(10, stateful=False,  return_sequences=False,  recurrent_dropout=0., kernel_regularizer=regularizers.L1L2(l1=0.0, l2=0.0))(x)
#     x = Dropout(0.8)(x)
    
    #assetCodeEmbedding = Embedding(3444, 10, embeddings_regularizer=regularizers.L1L2(l1=0.1, l2=0.1))(assetCodeInput)
    #assetCodeEmbedding = Reshape((10,)) (assetCodeEmbedding)
    #assetCodeEmbedding = Dense(10, activation='relu', kernel_regularizer=regularizers.L1L2(l1=0.01, l2=0.01))(assetCodeEmbedding)
    #x = Concatenate()([assetCodeEmbedding, x])

    x = Dense(5, activation="relu",  kernel_regularizer=regularizers.L1L2(l1=0.00, l2=0.00))(x)
    #x = Dense(10, activation="relu")(x)
    predictions = Dense(1, activation='sigmoid')(x)
    #predictions = Dense(1, activation='linear')(x)

    model = Model(inputs=[inputs, ], outputs=predictions)

    # model = Sequential()
    # model.add(LSTM(20, stateful=False,  return_sequences=True, batch_input_shape=(10, mytimestep, 1), kernel_regularizer=regularizers.L1L2(l1=0.0, l2=0.0)))
    # model.add(LSTM(20, stateful=False,  return_sequences=True, kernel_regularizer=regularizers.L1L2(l1=0.0, l2=0.0)))
    # model.add(LSTM(15, stateful=False,   kernel_regularizer=regularizers.L1L2(l1=0.0, l2=0.0)))
    # model.add(Dense(10))
    # #model.add(Dense(1, activation='linear'))
    # model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', metrics=['binary_crossentropy', 'accuracy'], optimizer='adam')
    #model.compile(loss='mse', metrics=['mse'], optimizer='sgd')
    return model

model1 = getLSTMModel()
model2 = getLSTMModel()
model3 = getLSTMModel()

model1.summary()


In [ ]:
# def get3dinput(indices, timestep, _mydata=mydata):
#     X = _mydata[indices]
#     y = myy[indices]
#     u = myu[indices]
#     d = myd[indices]
#     a = mya[indices]
    
    
#     XXlist = []
#     yylist= []
#     uulist = []
#     ddlist = []
#     aalist = []
#     for ii in range(timestep, X.shape[0]):
#         part = X[ii - timestep : ii]
#         XXlist.append(part)
#         yylist.append(y[ii])
#         uulist.append(u[ii])
#         ddlist.append(d[ii])
#         aalist.append(a[ii])
#     XX = np.array(XXlist)
#     yy = np.array(yylist)
#     uu = np.array(uulist)
#     dd = np.array(ddlist)
#     aa = np.array(aalist)
#     return XX,yy,uu,dd,aa

# # mydataoriginal = mydata.copy()
# # myyoriginal = myy.copy()

# # mydata = mydata[:,0,:]
# # myy = myy[:,0,:]
# print('getting 3d input train')
# train_X, train_y, train_u, train_d, train_a =  get3dinput(train_indices, mytimestep)
# print('getting 3d input val')
# val_X, val_y, val_u, val_d, val_a =  get3dinput(val_indices, mytimestep)

# # train_X = np.nan_to_num(train_X)
# # train_y = np.nan_to_num(train_y)

# # val_X = np.nan_to_num(val_X)
# # val_y = np.nan_to_num(val_y)

# # train_y = train_y.squeeze()
# # val_y = val_y.squeeze()
# # val_u = val_u.squeeze()

# #train_X = np.nan_to_num(train_X)
# # train_mask = np.isnan(train_y)
# # #train_y = np.nan_to_num(train_y)
# # train_u = np.nan_to_num(train_u)

# #val_X = np.nan_to_num(val_X)
# # val_mask = np.isnan(val_y)
# # #val_y = np.nan_to_num(val_y)
# # val_u = np.nan_to_num(val_u)

# train_y_bin = np.empty_like(train_y)
# val_y_bin = np.empty_like(val_y)
# train_y_bin[:] = 2
# val_y_bin[:] = 2

# train_y_bin[train_y > 0] = 1
# val_y_bin[val_y > 0] = 1

# train_y_bin[train_y < 0] = 0
# val_y_bin[val_y < 0] = 0

# train_y_bin_nn_input = train_y_bin.astype(float)
# #train_y_bin_nn_input[np.isnan(train_y)] = 0.5
# train_y_bin_nn_input = train_y_bin_nn_input.reshape(train_y_bin_nn_input.shape[0], train_y_bin_nn_input.shape[1])

# val_y_bin_nn_input = val_y_bin.astype(float)
# #val_y_bin_nn_input[np.isnan(val_y)] = 0.5
# val_y_bin_nn_input = val_y_bin_nn_input.reshape(val_y_bin_nn_input.shape[0], val_y_bin_nn_input.shape[1])

# print('done get3dinput train_X.shape', train_X.shape)
# print('train_a.shape', train_a.shape)
# print('val_a.shape', val_a.shape)

In [ ]:
# train_X = train_X[:,::2,:,:]
# print('train_X.shape', train_X.shape)
# val_X = val_X[:,::2,:,:]

In [ ]:
import gc
n = gc.collect()
print('gc collect called ', n)

# np.set_printoptions(edgeitems=20)
# print('train_X AAPL.O', train_X[0,:,3]) #3 is AAPL.O
# print('train_X AAPL.O', train_X[1,:,3]) #3 is AAPL.O
# print('train_X AAPL.O', train_X[2,:,3]) #3 is AAPL.O
# np.set_printoptions(edgeitems=3)

In [ ]:
# Lets print our model
# model1 = getModelAssetCode()
# model1.summary()

# model2 = getModelNoAssetCode()
# model2.summary()

In [ ]:
needed_cols = num_cols.copy()
needed_cols.extend(['time', 'assetCode'])
market_train = market_train[needed_cols]
history = market_train.iloc[-(3780 * (mytimestep)):,:]
# print('history intial shape', history.shape)
# print('history AA.N returnsOpenPrevMktres10', history[history.assetCode=='AA.N']['returnsOpenPrevMktres10'])
# print('history AA.N returnsOpenPrevMktres1', history[history.assetCode=='AA.N']['returnsOpenPrevMktres1'])
# print('history AA.N volume', history[history.assetCode=='AA.N']['volume'])
print('history', history.shape)
del market_train 
#del mydata, myy, myd, myu,
n = gc.collect()
print('gc collect called ', n)

[](http://) # Train NN model

In [ ]:
# print('reshaping')
# train_X_reshaped = np.vstack(np.split(train_X, 3780, axis=2))
# print('train_X_reshaped', train_X_reshaped.shape)
# del train_X
# train_X_reshaped = np.squeeze(train_X_reshaped)
# print('train_X_reshaped', train_X_reshaped.shape)

# train_a_reshaped = np.vstack(np.split(train_a, 3780, axis=1))
# train_a_reshaped = np.squeeLSTM...
# print('train_a_reshaped', train_a_reshaped.shape)

# train_y_reshaped = np.vstack(np.split(train_y_bin_nn_input, 3780, axis=1))
# print('train_y_reshaped', train_y_reshaped.shape)
# del train_y_bin_nn_input, train_a
# train_y_reshaped = np.squeeze(train_y_reshaped)
# print('train_y_reshaped', train_y_reshaped.shape)

# n = gc.collect()
# print('gc collect called ', n)

# train_non_nan_indexes = train_y_reshaped!=2
# print('train_non_nan_indexes', train_non_nan_indexes.shape)
# train_X_reshaped = train_X_reshaped[train_non_nan_indexes]
# train_y_reshaped = train_y_reshaped[train_non_nan_indexes]
# train_a_reshaped = train_a_reshaped[train_non_nan_indexes]

# del train_non_nan_indexes
# n = gc.collect()
# print('gc collect called ', n)

# val_X_reshaped =  np.vstack(np.split(val_X, 3780, axis=2))
# print('val_X_reshaped', val_X_reshaped.shape)
# del val_X
# val_X_reshaped = np.squeeze(val_X_reshaped)
# print('val_X_reshaped', val_X_reshaped.shape)

# val_a_reshaped = np.vstack(np.split(val_a, 3780, axis=1))
# val_a_reshaped = np.squeeze(val_a_reshaped)
# print('val_a_reshaped', val_a_reshaped.shape)

# val_y_reshaped = np.vstack(np.split(val_y_bin_nn_input, 3780, axis=1))
# del val_y_bin_nn_input, val_y_bin, val_a
# val_y_reshaped = np.squeeze(val_y_reshaped)
# val_non_nan_indexes = val_y_reshaped!=2
# val_X_reshaped = val_X_reshaped[val_non_nan_indexes]
# val_y_reshaped = val_y_reshaped[val_non_nan_indexes]
# val_a_reshaped = val_a_reshaped[val_non_nan_indexes]

# n = gc.collect()
# print('gc collect called ', n)



In [ ]:
# print('trim to be dividable by 1024')
# train_X_reshaped = train_X_reshaped[:((train_X_reshaped.shape[0]//1024) * 1024)]
# train_y_reshaped = train_y_reshaped[:((train_y_reshaped.shape[0]//1024) * 1024)]
# train_a_reshaped = train_a_reshaped[:((train_a_reshaped.shape[0]//1024) * 1024)]
# val_X_reshaped = val_X_reshaped[:((val_X_reshaped.shape[0]//1024) * 1024)]
# val_y_reshaped = val_y_reshaped[:((val_y_reshaped.shape[0]//1024) * 1024)]
# val_a_reshaped = val_a_reshaped[:((val_a_reshaped.shape[0]//1024) * 1024)]


In [ ]:
# print('np.nan_to_num')
# train_X_reshaped = np.nan_to_num(train_X_reshaped)
# val_X_reshaped = np.nan_to_num(val_X_reshaped)

In [ ]:
# train_unique_asset_ids = np.unique(train_a_reshaped)
# val_with_train_asset_ids = np.isin(val_a_reshaped, train_unique_asset_ids )
# print('val_with_train_asset_ids', val_with_train_asset_ids.shape)
# print('before val shapes', val_X_reshaped.shape, val_y_reshaped.shape, val_a_reshaped.shape)
# val_X_reshaped = val_X_reshaped[val_with_train_asset_ids]
# val_y_reshaped = val_y_reshaped[val_with_train_asset_ids]
# val_a_reshaped = val_a_reshaped[val_with_train_asset_ids]
# print('after val shapes', val_X_reshaped.shape, val_y_reshaped.shape, val_a_reshaped.shape)

In [ ]:
mydata[train_indices] = np.reshape(scaler.fit_transform(np.reshape(mydata[train_indices], (len(train_indices) * 3780, len(num_cols)))), (len(train_indices), 3780, len(num_cols)))
mydata[val_indices]   = np.reshape(scaler.transform    (np.reshape(mydata[val_indices],   (len(val_indices)   * 3780, len(num_cols)))), (len(val_indices),   3780, len(num_cols)))
history[num_cols] = scaler.transform(history[num_cols])

In [ ]:
my_batch_size=1024

def batch_generator(_mydata, _myy, batch_size=1024):
    day_index = mytimestep
    company_index = 0
    iii = 0
    
    while True:
        #print('day_index2', day_index)
        batch_X = np.zeros((batch_size, mytimestep, _mydata.shape[2]))
        batch_a = np.zeros((batch_size, mytimestep, 1))
        batch_y = np.zeros((batch_size, 1))
      
        ii = 0
        kk = 0
        while ii < batch_size:
           
            X_entrty = _mydata[day_index-mytimestep:day_index,company_index,:]
            y_entry = _myy[day_index, company_index]
            #print('day_index3', day_index, 'ii', ii, 'y_entry', y_entry, '(y_entry>0)', (y_entry>0))
            #print('y', y_entry[0], (y_entry[0] > 0 or  y_entry[0] < 0))
            #if (y_entry[0] > 0 or  y_entry[0] < 0) and np.isin(a_entry[0,0], train_unique_asset_ids):
            if (y_entry[0] > 0 or  y_entry[0] < 0):    
                X_entrty = np.nan_to_num(X_entrty)
                batch_X[ii,:,:] = X_entrty
                #batch_a[ii,:] = a_entry
                batch_y[ii,:] = 1 if y_entry>0 else 0
#                 if y_entry>0.2:
#                     #print('resetting y_entry from x to 1',y_entry)
#                     y_entry = 0.2
#                 if y_entry<-0.2:
#                     #print('resetting y_entry from x to -1',y_entry)
#                     y_entry = -0.2
#                 batch_y[ii,:] = y_entry
                ii = ii + 1
                iii = iii + 1
                
                
                #if kk % 1111 == 0:
                    #print('X_entrty', X_entrty)
                    #print('(y_entry>0)', (y_entry>0), y_entry, y_entry.shape)
            
            day_index = day_index + 1
            if day_index >= _mydata.shape[0]:
                day_index = mytimestep
                company_index = company_index + 1
            if company_index >= _mydata.shape[1]:
                #print('setting company_index = 0. iii=', iii)
                iii = 0
                company_index = 0
            
        yield [batch_X, ], batch_y

# train_X_reshaped = np.nan_to_num(train_X_reshaped)
# val_X_reshaped = np.nan_to_num(val_X_reshaped) 
    
train_gen = batch_generator(mydata[train_indices],  myy[train_indices], batch_size=my_batch_size)
valid_gen = batch_generator(mydata[val_indices],   myy[val_indices] , batch_size=my_batch_size)
latest_gen = batch_generator(mydata[latest_indices], myy[latest_indices], batch_size=my_batch_size)

In [ ]:
print('LSTM...')
final_model = True
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


# check_point = ModelCheckpoint('model.hdf5',verbose=True, save_best_only=True, monitor='val_binary_crossentropy')
# early_stop = EarlyStopping(patience=2,verbose=True, monitor='val_binary_crossentropy')

check_point2 = ModelCheckpoint('model.hdf5',verbose=True, save_best_only=True, monitor='val_binary_crossentropy')
early_stop2 = EarlyStopping(patience=4,verbose=True, monitor='val_binary_crossentropy')

# company_id1 = 3
# company_id2 = 3
# model.fit(np.expand_dims(train_X[:,:,company_id1], axis=2), train_y_bin_nn_input[:,company_id1], 
#            validation_data=(np.expand_dims(val_X[:,:,company_id2], axis=2), val_y_bin_nn_input[:,company_id2]),
#          train_unique_asset_ids  epochs=20, batrain_a_reshapedtch_size=10, verbose=2, shuffle=False)
    

# model.fit(train_X_reshaped, train_y_reshaped, 
#            validation_data=(print('train_X AAPL.O', train_X[0,:,3]) #3 is AAPL.O, val_y_reshaped),
#            epochs=10, batch_size=1024, verbose=2, shuffle=False,
#            callbacks=[check_point2, early_stop2])

train_ex_n = np.count_nonzero(~np.isnan(myy[train_indices_timestep]))
val_ex_n = np.count_nonzero(~np.isnan(myy[val_indices_timestep]))
latest_ex_n = np.count_nonzero(~np.isnan(myy[latest_indices]))

if final_model:
    print('training final model')
    model1.fit_generator(
        generator=latest_gen,
        epochs=10,
        verbose=2,
        steps_per_epoch=latest_ex_n // my_batch_size)
else:
    print('trainig test model')
    model1.fit_generator(
        generator=train_gen,
        epochs=20,
        verbose=2,
        steps_per_epoch=train_ex_n // my_batch_size,
        validation_data=valid_gen,
        validation_steps=val_ex_n // my_batch_size,
        callbacks=[check_point2, early_stop2])
    model1.load_weights('model.hdf5')

# model2.fit_generator(
#     generator=train_gen,
#     epochs=10,
#     verbose=2,
#     steps_per_epoch=train_ex_n // my_batch_size,
#     validation_data=valid_gen,
#     validation_steps=val_ex_n // my_batch_size,
#     callbacks=[check_point2, early_stop2])
# model2.load_weights('model.hdf5')

# model3.fit_generator(
#     generator=train_gen,
#     epochs=10,
#     verbose=2,
#     steps_per_epoch=train_ex_n // my_batch_size,
#     validation_data=valid_gen,
#     validation_steps=val_ex_n // my_batch_size,
#     callbacks=[check_point2, early_stop2])
# model3.load_weights('model.hdf5')



# model1.fit_generator(
#     generator=latest_gen,
#     epochs=4,
#     verbose=2,
#     steps_per_epoch=latest_ex_n // my_batch_size)

# model2.fit_generator(
#     generator=latest_gen,
#     epochs=4,
#     verbose=2,
#     steps_per_epoch=latest_ex_n // my_batch_size)

# model3.fit_generator(
#     generator=latest_gen,
#     epochs=4,
#     verbose=2,
#     steps_per_epoch=latest_ex_n // my_batch_size)


# code you want to evaluate


In [ ]:
gc.collect()
valid_gen = batch_generator(mydata[latest_indices],   myy[latest_indices], batch_size=my_batch_size)

valid_predictions = model1.predict_generator(valid_gen, steps=latest_ex_n // my_batch_size)
valid_predictions = valid_predictions
print('valid_predictions', valid_predictions.shape)

valid_gen2 = batch_generator(mydata[latest_indices],   myy[latest_indices], batch_size=my_batch_size)
val_y_raveled = np.zeros((valid_predictions.shape[0]))
for vvv in range(latest_ex_n // my_batch_size):
    _next = next(valid_gen2)
#     print('_next[1]', _next[1])
#     print('_next[1].shape', _next[1].shape)
    val_y_raveled[vvv * my_batch_size:(vvv + 1) * my_batch_size] = np.squeeze(_next[1])
print('-->', val_y_raveled.shape)
    

val_y_raveled_train = val_y_raveled[:800000]
val_y_raveled_val = val_y_raveled[800000:]
valid_predictions_train = valid_predictions[:800000]
valid_predictions_val = valid_predictions[800000:]

from sklearn.calibration import CalibratedClassifierCV
calibrator = CalibratedClassifierCV(method='isotonic', cv=10)
calibrator.fit(valid_predictions, val_y_raveled)

In [ ]:

# preds_list = []

# for _compId in range(mydata.shape[1]):
#     preds = model.predict(np.expand_dims(val_X[:,:,_compId], axis=2), batch_size=1)
#     preds_list.append(preds)
#     if _compId % 100 == 0:
#         print(_compId, 'preds.shape', preds.shape)
    
# endtime = timeit.default_timer()
# elapsed = endtime - start_time
# print('start_time', start_time, 'endtime', endtime, 'elapsed', elapsed)

# Evaluation of Validation Set

In [ ]:
# confidence_valid = np.concatenate(preds_list, axis=1)
# confidence_valid = confidence_valid * 2 -1
# val_y_bin_nn_input[val_y_bin_nn_input==0.5]=0
# print('accuracy_score', accuracy_score((confidence_valid>0).reshape((-1,)), val_y_bin_nn_input.reshape((-1,))))
# plt.hist(confidence_valid.reshape((-1,)), bins=20)
# plt.title("predicted confidence")
# plt.show()

In [ ]:
# # calculation of actual metric that is used to calculate final score
# val_y = np.nan_to_num(val_y)
# val_y = val_y.clip(-1,1) # get rid of outliers. Where do they come from??
# x_t_i = confidence_valid.reshape((-1,)) * val_y.reshape((-1,)) * val_u.reshape((-1,))
# data = {'day' : val_d.reshape((-1,)), 'x_t_i' : x_t_i}
# df = pd.DataFrame(data)
# x_t = df.groupby('day').sum().values.flatten()
# mean = np.mean(x_t)
# std = np.std(x_t)
# score_valid = mean / std
# print('mean', mean)
# print('std', std)
# print(score_valid)


# Prediction

In [ ]:
days = env.get_prediction_days()

In [ ]:
import gc
from sklearn.preprocessing import quantile_transform
from sklearn.preprocessing import power_transform
from sklearn.preprocessing import MinMaxScaler
minMaxScaler = MinMaxScaler(feature_range=(-1, 1))
n = gc.collect()
print('gc collect called ', n)

n_days = 0
prep_time = 0
prediction_time = 0
packaging_time = 0
predicted_confidences = np.array([])
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    n_days +=1
    print('******************************************')
    print('n_days', n_days, 'market_obs_df.shape', market_obs_df.shape)
    market_obs_df_rows = market_obs_df.shape[0]
    print('AA.N')
    print(market_obs_df[market_obs_df.assetCode=='AA.N'][['returnsOpenPrevMktres10', 'returnsOpenPrevMktres1', 'volume']])
    t = time.time()
        
    market_obs_df = enrich_df(market_obs_df)
    
    market_obs_df[num_cols] = scaler.transform(market_obs_df[num_cols])
    market_obs_df[num_cols] = market_obs_df[num_cols].fillna(0)
   
    market_obs_df = market_obs_df.set_index(['time', 'assetCode'], drop=False)
    market_obs_df = market_obs_df[needed_cols]
    
    print('market_obs_df', market_obs_df.head(5))
    print('concating')
    history = pd.concat([history, market_obs_df], ignore_index=False, sort=False)
    print('concated')
    
    (time_index, asset_index) = history.index.levels
    all_assetCodes = list(history.index.get_level_values(1).unique())
    new_index = pd.MultiIndex.from_product([time_index, all_assetCodes])
    history = history.reindex(new_index)
    
    history = history.iloc[-(len(all_assetCodes) * (mytimestep)):,:]

    (time_index, asset_index) = history.index.levels
    mytestdata = getMyData(history, _days=(mytimestep))
    print('mytestdata', mytestdata.shape)
    mytestdata = np.swapaxes(mytestdata, 0, 1)
    #print('mytestdata2:', mytestdata.shape)
    
    prep_time += time.time() - t
    
    t = time.time()
    mytestdata = np.nan_to_num(mytestdata)
    
#     AAN_index = list(history.index.get_level_values(1)).index('AA.N')
#     print('AAN mytestdata')
#     print(mytestdata[AAN_index, :, :])
    #if n_days == 1:
    #    firstAAN = mytestdata[AAN_index, :, :]
    #    firstAAN = np.expand_dims(firstAAN, axis=0)
    
    
    market_prediction = ((model1.predict(mytestdata, batch_size=1024))) # + (model2.predict(mytestdata, batch_size=1024) * 2 - 1) + (model3.predict(mytestdata, batch_size=1024) * 2 - 1)) /  3
    # market_prediction = np.expand_dims(ir.transform(np.squeeze(market_prediction)), axis=1)
    market_prediction = calibrator.predict_proba(market_prediction)[:,1]
    market_prediction = market_prediction * 2 - 1
    print('market_prediction', market_prediction.shape)
    print(market_prediction[:5])
    print('max market_prediction', np.max(market_prediction), 'min market_prediction', np.min(market_prediction))
    print('market_prediction>0', (market_prediction>0).sum(), 'market_prediction<0', (market_prediction<0).sum())
    prediction_time += time.time() - t
    market_prediction = np.squeeze(market_prediction)
    market_prediction = np.clip(market_prediction, -0.999, 0.999)
    # plt.hist(market_prediction, bins=100)
    predicted_confidences = np.concatenate((predicted_confidences, market_prediction))
    
    t = time.time()
    preds = pd.DataFrame({'assetCode':history.index.get_level_values(1).unique(),'confidence':market_prediction})
    # insert predictions to template
    #print('preds AA.N')
    #print( preds[preds.assetCode=='AA.N'])
    predictions_template_df = predictions_template_df.merge(preds,how='left').drop('confidenceValue',axis=1).fillna(0).rename(columns={'confidence':'confidenceValue'})
    #print('predictions_template_df:', predictions_template_df.head(5))
    #print('predictions_template_df.shape', predictions_template_df.shape)
    predictions_template_df['confidenceValue'] = np.squeeze(power_transform(np.expand_dims(predictions_template_df['confidenceValue'], axis=1), method='yeo-johnson'))
    
    
    ptd_mean = predictions_template_df['confidenceValue'].mean()
    print('ptd_mean', ptd_mean)
    predictions_template_df['confidenceValue'] = predictions_template_df['confidenceValue'] - ptd_mean
    
    predictions_template_sum = predictions_template_df['confidenceValue'].abs().sum()
    print('predictions_template_sum sum', predictions_template_sum)
    predictions_template_df['confidenceValue'] = predictions_template_df['confidenceValue'] / (predictions_template_sum / 100)
    #predictions_template_df['confidenceValue']  = np.squeeze(minMaxScaler.fit_transform(predictions_template_df['confidenceValue'].values.reshape((-1, 1))))
    
    print('predictions_template_df before submit', predictions_template_df.shape)
    print(predictions_template_df['confidenceValue'].head(5))
    print('predictions_template_dfmean', predictions_template_df['confidenceValue'].mean(), 'predictions_template_df sum', predictions_template_df['confidenceValue'].sum())
    env.predict(predictions_template_df)
    packaging_time += time.time() - t
10
env.write_submission_file()
total = prep_time + prediction_time + packaging_time
print(f'Preparing Data: {prep_time:.2f}s')
print(f'Making Predictions: {prediction_time:.2f}s')
print(f'Packing: {packaging_time:.2f}s')
print(f'Total: {total:.2f}s')

In [ ]:
# distribution of confidence as a sanity check: they should be distributed as above
plt.hist(predicted_confidences, bins='auto')
plt.title("predicted confidence")
plt.show()